In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GridSearchCV  
from sklearn.preprocessing import Imputer
from sklearn import linear_model
import sklearn.ensemble as en
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import time

rcParams['figure.figsize'] = 12, 4

xgbc = XGBClassifier(
 learning_rate =0.1,
 n_estimators=150,
 max_depth=5,
 min_child_weight=1,
 gamma=0,  
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
t = []

# clf = tree.DecisionTreeClassifier()
# clf = linear_model.LogisticRegression(C=10, solver='sag', penalty= 'l2' ,n_jobs=-1)
# clf = en.GradientBoostingClassifier(gb)
# clf = RandomForestClassifier(n_estimators=10, n_jobs=-1)
tfidf_vectorizer = TfidfVectorizer()
# param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
# tuned_parameters = {}
# clf = GridSearchCV(lr, tuned_parameters)
# GridSearchCV(cv=None,estimator=LogisticRegression(C=1.0, intercept_scaling=1, dual=False, fit_intercept=True,penalty='l2', tol=0.0001),param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

In [2]:
train_set = pd.read_csv("/home/mohitrbhardwaj/Desktop/kaggle/quora/train1.csv")
test_set = pd.read_csv("/home/mohitrbhardwaj/Desktop/kaggle/quora/test1.csv")

In [3]:
# len(test_set)

In [4]:
for x in train_set:
    print(x)

id
qid1
qid2
question1
question2
is_duplicate


In [6]:
res = tfidf_vectorizer.fit_transform(train_set['question1']+train_set['question2'])
tar = train_set['is_duplicate']

In [12]:
res.shape
# tar.shape

(404290, 87010)

In [13]:
# for x in res:
#     print(x)

In [14]:
# clf.fit(res,tar)

In [15]:
# test_res = tfidf_vectorizer.transform(test_set['question1']+test_set['question2'])

In [28]:
def modelfit(alg, dtrain,x,y,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
#         xgb_param = alg.get_xgb_params()
        xgb_param = {}
        xgb_param['objective'] = 'binary:logistic'
        xgb_param['eval_metric'] = 'logloss'
        xgb_param['eta'] = 0.02
        xgb_param['max_depth'] = 4
        xgtrain = xgb.DMatrix(x, y)
#         cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
#             metrics='auc', early_stopping_rounds=early_stopping_rounds)
#         alg.set_params(n_estimators=cvresult.shape[0])
        watchlist = [(xgtrain, 'train')]
        bst = xgb.train(xgb_param, xgtrain, 400, watchlist, early_stopping_rounds=5, verbose_eval=10)
    #Fit the algorithm on the data
#     alg.fit(x, y,eval_metric='logloss')
        
    #Predict training set:
#     dtrain_predictions = alg.predict(x)
#     dtrain_predprob = alg.predict_proba(x)[:,1]   
    #Print model report:
#     print("\nModel Report")
#     print("Accuracy : %.4g" % metrics.accuracy_score(y.values, dtrain_predictions))
#     print("log loss (Train): %f" % metrics.log_loss(y, dtrain_predprob))
#                     
#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')
#     print(feat_imp)

In [29]:
#     print(feat_imp)
# test_res.shape


In [30]:
start_time = time.time()
modelfit(xgbc, t,res,tar)
stop_time = time.time()
print(stop_time-start_time)

[0]	train-logloss:0.690625
Will train until train-logloss hasn't improved in 5 rounds.
[10]	train-logloss:0.670024
[20]	train-logloss:0.65472
[30]	train-logloss:0.643108
[40]	train-logloss:0.634153
[50]	train-logloss:0.626837
[60]	train-logloss:0.620831
[70]	train-logloss:0.615769
[80]	train-logloss:0.611394
[90]	train-logloss:0.607542
[100]	train-logloss:0.604112
[110]	train-logloss:0.600911
[120]	train-logloss:0.598059
[130]	train-logloss:0.595477
[140]	train-logloss:0.593198
[150]	train-logloss:0.59094
[160]	train-logloss:0.588801
[170]	train-logloss:0.586907
[180]	train-logloss:0.584845
[190]	train-logloss:0.583081
[200]	train-logloss:0.581437
[210]	train-logloss:0.579674
[220]	train-logloss:0.577971
[230]	train-logloss:0.576512
[240]	train-logloss:0.575086
[250]	train-logloss:0.573754
[260]	train-logloss:0.572384
[270]	train-logloss:0.571136
[280]	train-logloss:0.569912
[290]	train-logloss:0.568797
[300]	train-logloss:0.567649
[310]	train-logloss:0.566492
[320]	train-logloss:0.565

In [ ]:
# clf = xgbc

In [32]:
start_time = time.time()
i = 0
tot = []
for i in range(len(test_set['question1'][:100])):
    lis1=tfidf_vectorizer.transform([test_set['question1'][i]])+ tfidf_vectorizer.transform([test_set['question2'][i]])
    tot.append(bst.predict_proba(lis1.toarray()))
    i=i+1
stop_time = time.time()
print(stop_time-start_time)

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [ ]:
print(len(tot))

In [ ]:
tot = np.transpose(tot)
print(tot[1].shape)
xx=np.transpose(tot[1])
print(xx.shape)

In [ ]:
print(clf.score(tfidf_vectorizer.transform(train_set['question1'][13001:16000]+train_set['question2'][13001:16000]).toarray(),np.asarray(train_set['is_duplicate'][13001:16000])))

In [ ]:
sol = pd.DataFrame()
sol['test_id']=test_set['test_id']
print(sol['test_id'].shape)
# sol['question1']=test_set['question1'][:11]
# sol['question2']=test_set['question2'][:11]
print("yaha tk ho? gaya")
sol['is_duplicate']=xx
print("yeh bhi")
sol.to_csv("submission.csv",index=False)

In [ ]:
print(sol.head)